---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [0]:
from google.colab import drive
drive.mount('/data')

Drive already mounted at /data; to attempt to forcibly remount, call drive.mount("/data", force_remount=True).


In [0]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

!pip install xlrd

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [0]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [0]:
def get_list_of_university_towns():
  df = pd.read_csv('/data/My Drive/university_towns.txt', sep="\n", header=None, names=["university"])
#   df['university'] = df['university'].str.split('[').str[0]
  df['university'] = df['university'].str.split('(').str[0]
  data = pd.DataFrame([],columns=['State','RegionName'])
  df['compare'] = df['university'].str.contains('[edit]', regex=False)
  df['university'] = df['university'].str.split('[').str[0]
  s =''
  for index, row in df.iterrows():
    if row['compare']:
      s = row['university']
    else:
      df2 = pd.DataFrame([[s, row['university']]], columns=['State','RegionName'])
      data = data.append(df2, ignore_index=True)
    
  return data
get_list_of_university_towns()


,State,RegionName
0,Alabama,Auburn
1,Alabama,Florence
2,Alabama,Jacksonville
3,Alabama,Livingston
4,Alabama,Montevallo
5,Alabama,Troy
6,Alabama,Tuscaloosa
7,Alabama,Tuskegee
8,Alaska,Fairbanks
9,Arizona,Flagstaff


In [0]:
def  get_data_gdp():
  gdp = pd.read_excel('/data/My Drive/gdplev.xls', skiprows=range(219))
  gdp.drop(columns=['Unnamed: 0', 'Unnamed: 1', 'Unnamed: 2', 'Unnamed: 3', 'Unnamed: 7'], inplace=True)
  gdp.columns = ['time', 'gdp', 'chained2009']
  return gdp
  

In [0]:
def get_recession_start():
  gdp = get_data_gdp()
  
  counter = 0
  for i in range(len(gdp)):
    if(counter!=2):
        if(gdp['gdp'][i]> gdp['gdp'][i+1]):
            counter= counter+1
    else:
        break
  
  return gdp['time'][i]
get_recession_start()

'2008q4'

In [0]:
def get_recession_end():
  gdp = get_data_gdp()
  
  counter = 0
  for i in range(len(gdp)):
    if(counter!=2):
        if(gdp['gdp'][i]< gdp['gdp'][i+1]):
            counter= counter+1
    else:
        break
  
  return gdp['time'][i]
       
get_recession_end()

'2000q3'

In [0]:
def get_recession_bottom():
  gdp = get_data_gdp()
  
  start = get_recession_start()
  s_index = gdp[gdp.time == start].index[0]
  ends = get_recession_end()
  e_index = gdp[gdp.time == ends].index[0]

  index = s_index
  mins = ""
  while True:
        #print(index,gdp.iloc[index+1].GDP)
    if gdp.iloc[index+1].gdp < gdp.iloc[index].gdp:
      mins = gdp.iloc[index+1].time
      mins2 = index + 1
    else:
      pass
      index += 1
      if index >= e_index: break

    return mins
get_recession_bottom()

'2009q1'

In [0]:
def convert_housing_data_to_quarters():
  house = pd.read_csv("/data/My Drive/City_Zhvi_AllHomes.csv")
  ihouse = house.loc[:,"2000-01":"2016-08"]
  ihouse.columns = pd.to_datetime(ihouse.columns).to_period(freq="M")
  ghouse = ihouse.groupby(ihouse.columns.asfreq("Q"),axis=1).sum()
  house = (pd.merge(house.loc[:,"RegionID":"SizeRank"],ghouse,left_index=True,right_index=True,how="inner")
  .set_index(["State","RegionName"]).iloc[:,4:71])
  return house
convert_housing_data_to_quarters().size


718910

In [0]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    return "ANSWER"